In [1]:
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import category_encoders as ce
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from scipy import stats
from sklearn.model_selection import GridSearchCV

In [2]:
rawData = pd.read_csv("./Training.csv")
rawData.info()
rawData.describe()

C:\Users\tommy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (598,600,601) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50006 entries, 0 to 50005
Columns: 707 entries, Unnamed: 0 to indices
dtypes: float64(123), int64(550), object(34)
memory usage: 269.7+ MB


,Unnamed: 0,tot_page_views,tot_page_views_l30d,tot_page_views_l90d,tot_visits,tot_visits_l30d,tot_visits_l90d,log_in_events,internal_search_events,internal_campaign_events,...,ftr_dummy_idc_verticalsUnknown,tgt_first_date_hol_page_view,tgt_first_date_eval_page_view,tgt_first_date_seminar_page_view,tgt_first_date_webinar_page_view,tgt_first_date_whitepaper_download,tgt_first_date_any_download,tgt_more_than1,target,indices
count,50006.000000,50006.000000,50006.000000,50006.000000,50006.000000,50006.000000,50006.000000,50006.000000,50006.000000,50006.000000,...,50006.000000,50006.000000,50006.000000,50006.0,50006.000000,50006.000000,50006.000000,50006.000000,50006.000000,50006.0
mean,99934.576011,73.433288,13.383994,36.071491,12.854197,2.382554,6.428089,67.805303,1.133624,0.000080,...,0.732112,0.021757,0.000500,0.0,0.000460,0.000600,0.020578,0.002020,0.068432,1.0
std,57669.390240,760.254988,134.119192,377.833496,162.130164,28.586230,82.373655,678.805295,20.080458,0.008943,...,0.442863,0.145892,0.022354,0.0,0.021442,0.024486,0.141967,0.044897,0.514947,0.0
min,3.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,50072.750000,7.000000,0.000000,0.000000,1.000000,0.000000,0.000000,7.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,99849.500000,19.000000,0.000000,5.000000,3.000000,0.000000,1.000000,18.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
75%,149954.250000,55.000000,5.000000,25.000000,9.000000,1.000000,4.000000,51.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
max,199999.000000,110111.000000,18082.000000,55806.000000,24439.000000,4477.000000,12879.000000,103725.000000,3057.000000,1.000000,...,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,5.000000,1.0


### data cleaning

In [4]:
imputedData = rawData

In [5]:
def clean_and_mean_imputation(df):
 # """For numeric columns, imputes the mean for missing values
  #   and adds indicator variable for columns with >50% missing values
 # """
  numeric_cols = [col for col in df.columns if df[col].dtype in (int, float)]
  missing_vals = pd.DataFrame(df.isnull().sum().sort_values(ascending=False)/len(df)).reset_index()
  missing_vals.rename(columns={0:'missing','index':'colname'}, inplace=True)

  over_fifty = missing_vals[missing_vals['missing'] >= 0.5]['colname'].tolist()
  over_ninety = missing_vals[missing_vals['missing'] >= 0.9]['colname'].tolist()
  df.drop(columns = over_ninety, inplace=True)
  print(f'Dropped {len(over_ninety)} cols missing >90% of data')

  to_impute = [col for col in over_fifty if col not in over_ninety]
  for c in to_impute:
    df[c+"_imputed"] = [1 if x in [None, np.nan] else 0 for x in df[c]]

  output = df.apply(lambda x: x.fillna(x.mean()) if x.dtype in (int, float) else x.fillna("Missing Data"))
  return output

imputedData = clean_and_mean_imputation(imputedData)

Dropped 11 cols missing >90% of data


In [6]:
# drop all catvar columns as suggested from the EDA

imputedData = imputedData.drop(columns=["gu_city","gu_state","gu_sic_2_description","gu_sic_4_description","iso_country_dunssite",
"gu_iso_country_name","gu_country_geo"])


### remove target leakage

In [7]:
leakage = imputedData.corrwith(imputedData["target"])
leakage.sort_values(ascending=False)

target                                        1.000000
tgt_hol                                       0.959955
tgt_first_date_hol_page_view                  0.692242
tgt_first_date_any_download                   0.327053
tgt_download                                  0.324047
                                                ...   
total_prodA_booking_amount_imputed                 NaN
total_prodA_std_booking_amount_imputed             NaN
total_prodA_5_x_booking_amount_imputed             NaN
total_prodN_license_booking_amount_imputed         NaN
db_annualsales_imputed                             NaN
Length: 714, dtype: float64

In [8]:
imputedData = imputedData.drop(columns=["tgt_hol", "tgt_first_date_hol_page_view"])

In [9]:
target = imputedData["target"]

data = imputedData.drop(columns=["target"])


data["idc_verticals"] = data["idc_verticals"].astype("category")
data["db_industry"] = data["db_industry"].astype("category")
data["gu_emp_segment_desc"] = data["gu_emp_segment_desc"].astype("category")
data["gu_emp_segment"] = data["gu_emp_segment"].astype("category")
data["new_segment"] = data["new_segment"].astype("category")


### Splitting data

In [19]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3)

### encoding

In [20]:
encoder = ce.TargetEncoder()
training_dataset = encoder.fit_transform(X_train, y_train)

C:\Users\tommy\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


## Trying out random forest classifier

In [21]:
clf=RandomForestClassifier(class_weight = "balanced", random_state=0)
clf.fit(training_dataset,y_train)

RandomForestClassifier(class_weight='balanced', random_state=0)

In [22]:
encoder = ce.TargetEncoder()
testing_dataset = encoder.fit_transform(X_test, y_test)

y_pred_dt = clf.predict(testing_dataset)

C:\Users\tommy\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [23]:
class_names = [str(x) for x in clf.classes_]
print(classification_report(y_test, y_pred_dt, target_names=class_names))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     14583
           1       0.89      0.96      0.92       232
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         5
           4       0.86      0.19      0.32        31
           5       0.00      0.00      0.00       147

    accuracy                           0.99     15002
   macro avg       0.46      0.36      0.37     15002
weighted avg       0.98      0.99      0.98     15002



C:\Users\tommy\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
print(confusion_matrix(y_test, y_pred_dt))

[[14582     1     0     0     0     0]
 [   10   222     0     0     0     0]
 [    1     2     0     0     1     0]
 [    5     0     0     0     0     0]
 [    1    24     0     0     6     0]
 [  147     0     0     0     0     0]]


### hyper parameter tunning

In [25]:
params = {
    "max_depth" : [10,20,30],
    "n_estimators": [50,100,150,200]
}

search = GridSearchCV(clf, params, scoring="f1_macro", cv=5, verbose=1)
search = search.fit(training_dataset, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed: 15.4min finished


In [26]:
y_pred = search.predict(testing_dataset)
print(classification_report(y_test, y_pred_dt))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     14583
           1       0.89      0.96      0.92       232
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         5
           4       0.86      0.19      0.32        31
           5       0.00      0.00      0.00       147

    accuracy                           0.99     15002
   macro avg       0.46      0.36      0.37     15002
weighted avg       0.98      0.99      0.98     15002



C:\Users\tommy\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
grid_scores = search.cv_results_

In [32]:
# check out the grid search leaderboard

df = pd.DataFrame(grid_scores)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,4.841852,0.330496,0.136877,0.026486,10,50,"{'max_depth': 10, 'n_estimators': 50}",0.623408,0.428922,0.503537,0.603034,0.492659,0.530312,0.072622,1
1,9.345894,0.299912,0.217008,0.047663,10,100,"{'max_depth': 10, 'n_estimators': 100}",0.561265,0.418834,0.512417,0.594058,0.492616,0.515838,0.060237,3
2,13.702708,0.218879,0.291865,0.064998,10,150,"{'max_depth': 10, 'n_estimators': 150}",0.522687,0.530898,0.441422,0.612962,0.495487,0.520691,0.055751,2
3,18.778914,0.339021,0.383775,0.074023,10,200,"{'max_depth': 10, 'n_estimators': 200}",0.494950,0.423934,0.438160,0.575870,0.437893,0.474162,0.056411,4
4,6.268175,0.229440,0.159649,0.029311,20,50,"{'max_depth': 20, 'n_estimators': 50}",0.320602,0.349996,0.336882,0.422143,0.340609,0.354046,0.035348,9
5,13.299051,0.291130,0.303380,0.015288,20,100,"{'max_depth': 20, 'n_estimators': 100}",0.320707,0.322998,0.330162,0.418493,0.320182,0.342508,0.038159,10
6,19.882436,0.966615,0.386127,0.042219,20,150,"{'max_depth': 20, 'n_estimators': 150}",0.322321,0.320859,0.320182,0.417981,0.321768,0.340622,0.038686,11
7,25.986710,0.661163,0.552347,0.040591,20,200,"{'max_depth': 20, 'n_estimators': 200}",0.321529,0.320859,0.321756,0.417297,0.320985,0.340485,0.038407,12
8,6.474292,0.179253,0.170980,0.035770,30,50,"{'max_depth': 30, 'n_estimators': 50}",0.323693,0.318593,0.368058,0.349867,0.426926,0.357428,0.039106,8
9,13.270064,0.551770,0.301938,0.008673,30,100,"{'max_depth': 30, 'n_estimators': 100}",0.323693,0.318444,0.319385,0.460407,0.402602,0.364906,0.057395,7


In [40]:
print(confusion_matrix(y_test, y_pred_dt))

[[14578     0     0     0     0     0]
 [    4   237     0     0     0     0]
 [    1     0     0     0     0     0]
 [    5     1     0     0     0     0]
 [    2    25     0     0     2     0]
 [  147     0     0     0     0     0]]


# Binary target column

In [41]:
binaryTarget = target

binaryTarget = np.where(binaryTarget > 1, 1, 0)
target0 = binaryTarget[binaryTarget==0]
target1 = binaryTarget[binaryTarget==1]

In [42]:
stats.describe(target0)
stats.describe(target1)

DescribeResult(nobs=49447, minmax=(0, 0), mean=0.0, variance=0.0, skewness=0.0, kurtosis=-3.0)

DescribeResult(nobs=559, minmax=(1, 1), mean=1.0, variance=0.0, skewness=0.0, kurtosis=-3.0)

In [43]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(data, binaryTarget, test_size=0.3)

In [44]:
encoder = ce.TargetEncoder()
training_dataset2 = encoder.fit_transform(X_train2, y_train2)

C:\Users\tommy\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [45]:
clf2=RandomForestClassifier(max_depth=5, class_weight = "balanced", random_state=0)
clf2.fit(training_dataset2,y_train2)

RandomForestClassifier(class_weight='balanced', max_depth=5, random_state=0)

In [46]:
encoder = ce.TargetEncoder()
testing_dataset2 = encoder.fit_transform(X_test2, y_test2)

y_pred_dt2 = clf2.predict(testing_dataset2)

C:\Users\tommy\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [47]:
class_names2 = [str(x) for x in clf2.classes_]
print(classification_report(y_test2, y_pred_dt2, target_names=class_names2))

              precision    recall  f1-score   support

           0       1.00      0.97      0.99     14839
           1       0.25      0.82      0.38       163

    accuracy                           0.97     15002
   macro avg       0.62      0.89      0.68     15002
weighted avg       0.99      0.97      0.98     15002



In [48]:
print(confusion_matrix(y_test2, y_pred_dt2))

[[14430   409]
 [   30   133]]


### hpyerparameter tunning for binary target

In [49]:
search2 = GridSearchCV(clf2, params, scoring="f1_macro", cv=5, verbose=1)
search2 = search2.fit(training_dataset2, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [50]:
y_pred2 = search2.predict(testing_dataset2)
print(classification_report(y_test2, y_pred_dt2))

              precision    recall  f1-score   support

           0       1.00      0.97      0.99     14839
           1       0.25      0.82      0.38       163

    accuracy                           0.97     15002
   macro avg       0.62      0.89      0.68     15002
weighted avg       0.99      0.97      0.98     15002



In [51]:
grid_scores2 = search2.cv_results_
df2 = pd.DataFrame(grid_scores2)
df2

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2.561941,0.117398,0.073542,0.008651,10,50,"{'max_depth': 10, 'n_estimators': 50}",0.158258,0.165331,0.156317,0.160666,0.157694,0.159653,0.003168,12
1,5.799139,0.302088,0.137120,0.015938,10,100,"{'max_depth': 10, 'n_estimators': 100}",0.162239,0.166069,0.159849,0.162871,0.157488,0.161703,0.002896,11
2,7.985681,0.385421,0.176514,0.014392,10,150,"{'max_depth': 10, 'n_estimators': 150}",0.163729,0.165255,0.163890,0.164006,0.157768,0.162930,0.002637,10
3,10.248551,0.124791,0.221889,0.022883,10,200,"{'max_depth': 10, 'n_estimators': 200}",0.165754,0.164799,0.163985,0.161770,0.159465,0.163155,0.002267,9
4,4.815694,0.256519,0.111174,0.016347,20,50,"{'max_depth': 20, 'n_estimators': 50}",0.164472,0.164460,0.164460,0.164460,0.164471,0.164464,0.000006,5
5,9.174365,0.111348,0.181508,0.003725,20,100,"{'max_depth': 20, 'n_estimators': 100}",0.164472,0.164472,0.164460,0.164460,0.164471,0.164467,0.000006,1
6,13.741819,0.233658,0.265426,0.025001,20,150,"{'max_depth': 20, 'n_estimators': 150}",0.164472,0.164472,0.164460,0.164447,0.164471,0.164464,0.000010,6
7,18.165733,0.201797,0.335651,0.011118,20,200,"{'max_depth': 20, 'n_estimators': 200}",0.164472,0.164472,0.164460,0.164447,0.164471,0.164464,0.000010,6
8,5.188426,0.059769,0.119140,0.006732,30,50,"{'max_depth': 30, 'n_estimators': 50}",0.164472,0.164472,0.164460,0.164460,0.164471,0.164467,0.000006,1
9,10.437815,0.456916,0.207452,0.016090,30,100,"{'max_depth': 30, 'n_estimators': 100}",0.164472,0.164472,0.164460,0.164460,0.164471,0.164467,0.000006,1


In [52]:
print(confusion_matrix(y_test2, y_pred_dt2))

[[14430   409]
 [   30   133]]


## Summary:


Since we have an imbalance class, we should be looking at the marco avg intead of weighted average

multi-class: 0.55
binary class: 0.72

We can see that with binary target, the accuracy had increase significantly for the same dataset